In [1]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.datasets import fetch_openml

from dense_feature_mixer import DenseFeatureMixer
from dense_feature_mixer.compose import ColumnTransformerWithNames


X, y = fetch_openml("titanic", version=1, return_X_y=True, as_frame=True)
X = X.drop(["name", "ticket", "cabin", "boat", "body"], axis=1)
y = y.reindex(X.index).astype(int)
numeric_vars = ["age", "fare"]
categorical_vars = ["sex", "pclass", "parch", "embarked"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [2]:
dfm = DenseFeatureMixer(task="classification", encode=True, unknown_category=999)
num_pipe = make_pipeline(SimpleImputer(strategy="mean"), StandardScaler())
cat_pipe = make_pipeline(SimpleImputer(strategy="most_frequent"), dfm)
col_transformer = ColumnTransformer([("num_transformer", num_pipe, numeric_vars),
                                     ("cat_transformer", cat_pipe, categorical_vars)])

pipe = make_pipeline(col_transformer,
                     LogisticRegression())

cv_score = cross_val_score(pipe, X_train, y_train, cv=3)
print(f"Mean score: {cv_score.mean():.2f} ({cv_score.std():.2f})")

Mean score: 0.78 (0.04)


In [6]:
dfm = DenseFeatureMixer(task="classification", categorical_vars=categorical_vars, 
                        numeric_vars=numeric_vars, encode=True, unknown_category=999)
num_pipe = make_pipeline(SimpleImputer(strategy="mean"), StandardScaler())
cat_transformer = SimpleImputer(strategy="most_frequent")
col_transformer = ColumnTransformerWithNames([("num_transformer", num_pipe, numeric_vars),
                                              ("cat_transformer", cat_transformer, categorical_vars)])

pipe = make_pipeline(col_transformer,
                     dfm,
                     LogisticRegression())

cv_score = cross_val_score(pipe, X_train, y_train, cv=3)
print(f"Mean score: {cv_score.mean():.2f} ({cv_score.std():.2f})")

Mean score: nan (nan)


c:\Users\41801990\Documents\Code\Dense-Feature-Mixer\dense_feature_mixer\core.py:135: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  X_copy[self._categorical_vars] = self._ordinal_encoder.fit_transform(X_copy[self._categorical_vars])
c:\Users\41801990\Documents\Code\Dense-Feature-Mixer\dense_feature_mixer\core.py:135: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  X_copy[self._categorical_vars] = self._ordinal_encoder.fit_transform(X_copy[self._categorical_vars])
c:\Users\41801990\Documents\Code\Dense-Feature-Mixer\dense_feature_mixer\core.